In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# read csv
data = pd.read_csv("./../../datasets/All.csv")
data.index = pd.DatetimeIndex(data.DateTime)
data.head()

,NO2-ug-per-m3,Cars-Direction-1,Cars-Direction-2,Cars-Total,Day-Of-Week,Monday,Tuesday,Wednesday,Thursday,Friday,...,Number of vessels,DateTime,DaySin,DayCos,WeekSin,WeekCos,MonthSin,MonthCos,YearSin,YearCos
DateTime,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,19.0,36,62,98,2,0,1,0,0,0,...,6,2019-01-01 00:00:00,0.000000,1.000000,0.781831,0.62349,0.201299,0.97953,0.017213,0.999852
2019-01-01 01:00:00,11.0,109,112,221,2,0,1,0,0,0,...,4,2019-01-01 01:00:00,0.258819,0.965926,0.781831,0.62349,0.201299,0.97953,0.017213,0.999852
2019-01-01 02:00:00,9.0,97,60,157,2,0,1,0,0,0,...,2,2019-01-01 02:00:00,0.500000,0.866025,0.781831,0.62349,0.201299,0.97953,0.017213,0.999852
2019-01-01 03:00:00,7.0,60,44,104,2,0,1,0,0,0,...,4,2019-01-01 03:00:00,0.707107,0.707107,0.781831,0.62349,0.201299,0.97953,0.017213,0.999852
2019-01-01 04:00:00,8.0,44,33,77,2,0,1,0,0,0,...,3,2019-01-01 04:00:00,0.866025,0.500000,0.781831,0.62349,0.201299,0.97953,0.017213,0.999852


In [3]:
X = data[["Cars-Total", "diesel", "wind_meters_per_second"]].to_numpy()
Y = data["NO2-ug-per-m3"].to_numpy()
(X.shape, Y.shape)

((8052, 3), (8052,))

In [4]:
display(data["Cars-Total"].max())
display(data["diesel"].max())
display(data["wind_meters_per_second"].max())
display(data["NO2-ug-per-m3"].max())

1691

1.413516129

13.9

99.0

In [5]:
x_scale_factors = [1000.0, 10.0, 10.0]
y_scale_factor = 100.0

X[:, 0] /= x_scale_factors[0]
X[:, 1] /= x_scale_factors[1]
X[:, 2] /= x_scale_factors[2]
Y = Y / y_scale_factor

In [7]:
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow.keras import Model

print("TensorFlow version:", tf.__version__)

model = tf.keras.Sequential()
for i in range(2):
    model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation="linear"))

model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=['mean_absolute_error'])
model.fit(X, Y, batch_size=64, epochs=10, validation_split=0.2)



TensorFlow version: 2.3.0
Epoch 1/10
101/101 [==============================] - 0s 3ms/step - loss: 0.1223 - mean_absolute_error: 0.2737 - val_loss: 0.0415 - val_mean_absolute_error: 0.1710
Epoch 2/10
101/101 [==============================] - 0s 1ms/step - loss: 0.0466 - mean_absolute_error: 0.1800 - val_loss: 0.0351 - val_mean_absolute_error: 0.1579
Epoch 3/10
101/101 [==============================] - 0s 1ms/step - loss: 0.0423 - mean_absolute_error: 0.1715 - val_loss: 0.0284 - val_mean_absolute_error: 0.1386
Epoch 4/10
101/101 [==============================] - 0s 1ms/step - loss: 0.0397 - mean_absolute_error: 0.1648 - val_loss: 0.0260 - val_mean_absolute_error: 0.1312
Epoch 5/10
101/101 [==============================] - 0s 1ms/step - loss: 0.0378 - mean_absolute_error: 0.1599 - val_loss: 0.0247 - val_mean_absolute_error: 0.1282
Epoch 6/10
101/101 [==============================] - 0s 1ms/step - loss: 0.0366 - mean_absolute_error: 0.1560 - val_loss: 0.0236 - val_mean_absolute_erro